In [14]:
import pandas as pd
import torch
import chromadb
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

df = pd.read_csv('C:\\Users\\Leo\\code\\ch07_트랜스포머_RAG\\test03\\data\\data.csv', encoding='utf8')
df.head(10)


,text
0,Etching 공정 전에는 반드시 세정 공정이 완료되어야 합니다.
1,PM 장비의 필터는 주 1회 정기적으로 교체해야 합니다.
2,웨이퍼 투입 전 챔버 내부의 온도 안정화가 필요합니다.
3,불량률이 2%를 초과할 경우 원인 분석 보고서를 제출해야 합니다.
4,클린룸 입장 전에는 반드시 정전기 방지복을 착용해야 합니다.
5,포토 공정 시 PR 코팅 두께는 1.5μm 이상 유지해야 합니다.
6,검사 장비는 매일 초기화 후 기능 점검을 실시합니다.
7,주간 생산 계획은 매주 월요일 오전 9시에 확정됩니다.
8,X선 검사 장비는 비정상 신호 발생 시 즉시 사용 중지합니다.
9,로더 설비의 진공 펌프는 월 1회 이상 윤활 상태를 점검합니다.


In [ ]:
sc_list = df['text'].tolist()
sc_list
from sentence_transformers import SentenceTransformer
embdedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
ids_list = []
ebd_list = []
text_list = []

for i, sc in enumerate(sc_list):
    doc_id = f'doc_{i+1}'
    ids_list.append(doc_id)
    text_list.append({"text": sc})

    embedding = embdedding_model.encode(sc).tolist()
    ebd_list.append(embedding)


In [16]:
client = chromadb.PersistentClient(path="./chromaDB")
collection = client.get_or_create_collection("company_doc")

collection.add(
    ids = ids_list, 
    embeddings=ebd_list,
    metadatas=text_list
)

In [17]:
question = "MES 공정 로그의 보관 기간은?"
question_embedding = embdedding_model.encode(question).tolist()
print(question_embedding)

[-0.005188596434891224, 0.08835329860448837, 0.03435646742582321, 0.003760016057640314, 0.03819876164197922, -0.14238812029361725, 0.13218359649181366, 0.1268777847290039, -0.01978047378361225, 0.0097413444891572, 0.03453388437628746, 0.016829850152134895, 0.05287565290927887, -0.004264624789357185, -0.0009695439948700368, -0.0427466481924057, 0.05801384896039963, -0.0391794815659523, -0.0631851851940155, -0.056479889899492264, 0.004144270904362202, -0.024508917704224586, -0.010005838237702847, 0.05418601632118225, -0.08088277280330658, -0.01259471196681261, 0.0075879087671637535, 0.03861331194639206, 0.05158284306526184, -0.021537108346819878, -0.022150322794914246, 0.0729685127735138, 0.090932197868824, -0.007085601333528757, -0.0654262974858284, 0.05937549099326134, 0.04323423653841019, -0.05415506288409233, -0.029932910576462746, 0.046879347413778305, -0.14287613332271576, -0.1027441993355751, 0.09817223995923996, -0.07965843379497528, 0.0804021805524826, -0.008331004530191422, -0.

In [18]:
result = collection.query(query_embeddings=[question_embedding], n_results=1)
retrieved_text = result['metadatas'][0][0]['text']
print(f"선택된 문서: {retrieved_text}")

선택된 문서: MES 시스템에 기록된 공정 로그는 6개월간 보관됩니다.


In [ ]:
qa_model_name = "monologg/koelectra-base-v3-finetuned-korquad"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model_name = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# QA 모델에 질문과 문서 입력
imputs = qa_tokenizer(question, retrieved_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
with torch.no_grad():
    outputs = qa_model_name(**imputs)

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1

answer = qa_tokenizer.decode(imputs["input_ids"][0][start_idx:end_idx], skip_special_tokens=True)

print(f"질문: {question}")
print(f"답변: {answer}")


질문: MES 공정 로그의 보관 기간은?
답변: 6개월
